In [3]:
import csv
import folium
from folium.plugins import MarkerCluster

# 1) Nom du fichier CSV (même dossier que le notebook)
chemin = "experimentations_5G.csv"

# 2) >>>>> ADAPTE ICI LES NOMS DE COLONNES SI BESOIN <<<<<
# Mets exactement ce que tu as vu avec print(lecteur.fieldnames)
COL_LAT = "Latitude"              # colonne latitude (ex : "Latitude" ou "latitude")
COL_LON = "Longitude"             # colonne longitude (ex : "Longitude" ou "longitude")
COL_REGION = "Région"             # colonne région
COL_EXP = "Expérimentateur"       # colonne expérimentateur
COL_BANDE = "Bande de fréquences" # colonne bande de fréquences (si elle existe)

# 3) On lit le fichier et on prépare la liste des points
points = []

with open(chemin, encoding="cp1252", newline="") as f:
    lecteur = csv.DictReader(f, delimiter=";")
    print("Colonnes trouvées :", lecteur.fieldnames)   # utile pour vérifier les noms

    for ligne in lecteur:
        lat_str = (ligne.get(COL_LAT, "") or "").strip()
        lon_str = (ligne.get(COL_LON, "") or "").strip()

        # On saute les lignes sans coordonnées
        if lat_str == "" or lon_str == "":
            continue

        # On remplace la virgule par un point pour faire un float
        lat_str = lat_str.replace(",", ".")
        lon_str = lon_str.replace(",", ".")

        try:
            lat = float(lat_str)
            lon = float(lon_str)
        except ValueError:
            # Si la conversion échoue, on ignore cette ligne
            continue

        region = (ligne.get(COL_REGION, "") or "").strip()
        exp = (ligne.get(COL_EXP, "") or "").strip()
        bande = (ligne.get(COL_BANDE, "") or "").strip()

        popup = f"""
        <b>Région :</b> {region}<br>
        <b>Expérimentateur :</b> {exp}<br>
        <b>Bande de fréquences :</b> {bande}<br>
        """

        # On ajoute CE SITE à la liste des points
        points.append((lat, lon, popup))

print("Nombre de sites avec coordonnées :", len(points))

# 4) Si aucun site trouvé, on s'arrête
if not points:
    print("Aucun point trouvé. Vérifie les noms de colonnes COL_LAT et COL_LON.")
else:
    # Centre de la carte = moyenne des latitudes et longitudes
    lat_moy = sum(p[0] for p in points) / len(points)
    lon_moy = sum(p[1] for p in points) / len(points)

    # Création de la carte
    m = folium.Map(location=[lat_moy, lon_moy], zoom_start=6, tiles="OpenStreetMap")

    # Cluster pour que TOUS les sites soient visibles, même s'ils sont nombreux
    cluster = MarkerCluster().add_to(m)

    # Ajout d'UN POINT par site
    for lat, lon, popup in points:
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(popup, max_width=300)
        ).add_to(cluster)

    # Affiche la carte dans Jupyter
    m


Colonnes trouvées : ['Expérimentateur', 'Bande de fréquences', 'Fréquences attribuées (limite basse)', 'Fréquences attribuées (limite haute)', "Numéro de la décision d'autorisation de l'Arcep", "Lien de la décision d'autorisation de l'Arcep", 'Latitude', 'Longitude', 'Code INSEE', 'Commune', 'Département', 'Région', 'Début', 'Fin', 'Description', 'Techno - Massive MIMO', 'Techno - Beamforming/beamtracking', 'Techno - Duplexage temporel (mode TDD)', 'Techno - Mode de fonctionnement NSA (Non Stand Alone)', 'Techno - Mode de fonctionnement SA (Stand Alone)', 'Techno - Synchronisation de réseaux', 'Techno - Network slicing', 'Techno - Small cells', 'Techno - Accès dynamique au spectre', 'Techno - 5G, 6G…', 'Usage - Mobilité connectée', 'Usage - Internet des objets', 'Usage - Ville intelligente', 'Usage - Réalité virtuelle', 'Usage - Télémédecine', 'Usage - Industrie du futur', 'Usage - Autre']
Nombre de sites avec coordonnées : 116


In [4]:
m.save("carte_experimentations_5G.html")


In [22]:
import csv
import folium

# Nom du fichier CSV (dans le même dossier que le notebook)
fichier_csv = "experimentations_5G.csv"

# Noms des colonnes (d'après ce que tu as affiché)
col_lat = "Latitude"
col_lon = "Longitude"
col_region = "Région"
col_exp = "Expérimentateur"
col_bande = "Bande de fréquences"

# Liste pour stocker les points (latitude, longitude, texte du popup)
liste_points = []

with open(fichier_csv, encoding="cp1252", newline="") as f:
    lecteur = csv.DictReader(f, delimiter=";")

    for ligne in lecteur:
        # récupération des coordonnées
        lat_texte = ligne[col_lat].strip()
        lon_texte = ligne[col_lon].strip()

        if lat_texte == "" or lon_texte == "":
            continue  # on saute si coordonnée manquante

        lat_texte = lat_texte.replace(",", ".")
        lon_texte = lon_texte.replace(",", ".")

        try:
            latitude = float(lat_texte)
            longitude = float(lon_texte)
        except ValueError:
            continue  # on saute si ce n'est pas un nombre

        region = ligne[col_region].strip()
        exp = ligne[col_exp].strip()
        bande = ligne[col_bande].strip()

        # petit texte qui s'affiche quand on clique sur le point
        popup = (
            f"<b>Région :</b> {region}<br>"
            f"<b>Expérimentateur :</b> {exp}<br>"
            f"<b>Bande de fréquences :</b> {bande}"
        )

        liste_points.append((latitude, longitude, popup))

print("Nombre de sites pris en compte :", len(liste_points))

if len(liste_points) == 0:
    print("Aucun point trouvé, vérifier les colonnes latitude/longitude.")
else:
    # calcul d'un centre approximatif de la carte
    lat_moy = sum(p[0] for p in liste_points) / len(liste_points)
    lon_moy = sum(p[1] for p in liste_points) / len(liste_points)

    # création de la carte
    carte = folium.Map(location=[lat_moy, lon_moy], zoom_start=6)

    # ajout d'un marqueur pour chaque site
    for (lat, lon, texte_popup) in liste_points:
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(texte_popup, max_width=250)
        ).add_to(carte)

    # affichage dans le notebook
    carte


Nombre de sites pris en compte : 116


In [23]:
carte.save("carte_exps_5G.html")
print("Fichier généré : carte_exps_5G.html")


Fichier généré : carte_exps_5G.html
